In [27]:
# !pip install langchain google-generativeai langchain-google-genai streamlit chromadb pypdf

In [2]:
import os
import google.generativeai as genai
import pandas as pd

GOOGLE_API_KEY = "YOUR-KEY"
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
genai.configure(api_key = os.environ['GOOGLE_API_KEY'])

C:\Users\shubh\anaconda3\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('data.csv')

## LLMChain

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [12]:
llm = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0.9, google_api_key=GOOGLE_API_KEY)

In [13]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

In [14]:
chain = LLMChain(llm=llm, prompt=prompt)

In [17]:
product = "Sweets"
print(chain.run(product))

* Sweet Delights
* Sugar Rush
* Candy Cove
* Sweet Sensations
* Confectionary Creations
* Sugar Bliss
* Sweet Tooth Emporium
* Candy Paradise
* Dessert Dreams
* Treat Factory


## SimpleSequentialChain

In [18]:
from langchain.chains import SimpleSequentialChain

In [22]:
llm = llm = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0.9, google_api_key=GOOGLE_API_KEY)

first_prompt = ChatPromptTemplate.from_template(
    "What is one best name to describe a company that makes {product}?"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)

second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company: {company}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

In [23]:
print(overall_simple_chain.run(product))



> Entering new SimpleSequentialChain chain...
Sugarly Delights
Sugarly Delights: Indulge in handcrafted sweet treats, from decadent cupcakes to beautifully designed cakes, made with passion and creativity to satisfy your sweet cravings.

> Finished chain.
Sugarly Delights: Indulge in handcrafted sweet treats, from decadent cupcakes to beautifully designed cakes, made with passion and creativity to satisfy your sweet cravings.


## SequentialChain

In [25]:
from langchain.chains import SequentialChain
llm = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0.9, google_api_key=GOOGLE_API_KEY)

In [40]:
# First prompt template
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:\n\n{Review}"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="englsih_review")

In [42]:
# Second prompt template
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following reivew in one sentence:\n\n {englsih_review}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

In [43]:
# Third prompt template
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key='language')

In [44]:
# Fourth prompt template
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following summary in the specified language:\n\nSummary: {summary}\n\nLanguage: {language}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key='followup_message')

In [47]:
overall_chain = SequentialChain(
    chains = [chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["englsih_review", "summary", "followup_message"],
    verbose=True,
)

In [48]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'englsih_review': 'I find the taste mediocre. The foam does not hold up, which is weird. I buy the same ones at the store and the taste is so much better...\nOld batch or fake!?',
 'summary': 'Reviewer finds the taste and foam quality of the pretzel buns mediocre and different from the ones bought at the store, raising concerns about the freshness or authenticity of the product.',
 'followup_message': "Le critique trouve que le goût et la qualité de la mousse des petits pains bretzels sont médiocres et différents de ceux achetés en magasin, ce qui soulève des inquiétudes quant à la fraîcheur ou à l'authenticité du produit."}

## Router Chain

In [5]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [6]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

In [7]:
from langchain.chains import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

llm = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0, google_api_key=GOOGLE_API_KEY)

In [8]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info['name']
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [9]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_prompt = LLMChain(llm=llm, prompt=default_prompt)

In [10]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revisingit will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>


In [15]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=['input'],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain,
                        destination_chains=destination_chains,
                        default_chain=default_chain, verbose=True)

In [20]:
response = chain.invoke('why can not we travel more than speed of light?')

In [26]:
print(response)

{'input': 'why can not we travel more than speed of light?', 'text': "The speed of light is the fastest possible speed in the universe, and it is a fundamental constant of nature. It is the speed at which all electromagnetic radiation, including visible light, travels. The speed of light is approximately 299,792,458 meters per second (186,282 miles per second).\n\nThere are several reasons why we cannot travel faster than the speed of light.\n\n* **Time dilation:** As an object approaches the speed of light, time slows down for that object. This means that it would take an infinite amount of time for an object to reach the speed of light.\n* **Length contraction:** As an object approaches the speed of light, it also contracts in length. This means that the distance between two points would become infinitely small at the speed of light.\n* **Infinite energy:** It would take an infinite amount of energy to accelerate an object to the speed of light. This is because the energy required to